In [ ]:
import torch, math
from torch import nn
from torch.utils.data import Dataset, DataLoader
from transformers import AutoImageProcessor, Siglip2ForImageClassification

# 1) Чекпоинт FixRes (фикс. 384x384) — удобен как старт
model_id = "google/siglip2-large-patch16-384"  # FixRes вариант
processor = AutoImageProcessor.from_pretrained(model_id)  # даст resize/normalize, mean/std/size
# Веса энкодера + НОВАЯ голова классификации (num_labels=2):
model = Siglip2ForImageClassification.from_pretrained(
    model_id,
    num_labels=2,
    ignore_mismatched_sizes=True,  # создаст новую голову нужного размера
)

device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

# 2) Заморозим всё, кроме головы (линейный пробинг)
for name, p in model.named_parameters():
    p.requires_grad = ("classifier" in name)  # у HF-классификаторов голова обычно называется "classifier"

# 3) Датасет с использованием processor (он вернёт корректные pixel_values)
class ForestDataset(Dataset):
    def __init__(self, paths, labels):  # labels: 0=blank, 1=animal
        self.paths = paths
        self.labels = labels

    def __len__(self): return len(self.paths)

    def __getitem__(self, idx):
        from PIL import Image
        img = Image.open(self.paths[idx]).convert("RGB")
        enc = processor(images=img, return_tensors="pt")
        # enc["pixel_values"]: (1, C, H, W) -> уберём размерность 0
        return {"pixel_values": enc["pixel_values"].squeeze(0),
                "labels": torch.tensor(self.labels[idx], dtype=torch.long)}

def collate_fn(batch):
    pixel_values = torch.stack([b["pixel_values"] for b in batch], dim=0)
    labels = torch.stack([b["labels"] for b in batch], dim=0)
    return {"pixel_values": pixel_values, "labels": labels}

# Пример: соберите свои train/val списки файлов и меток
train_ds = ForestDataset(train_paths, train_labels)
val_ds   = ForestDataset(val_paths,   val_labels)
train_loader = DataLoader(train_ds, batch_size=16, shuffle=True, num_workers=4, collate_fn=collate_fn)
val_loader   = DataLoader(val_ds,   batch_size=32, shuffle=False, num_workers=4, collate_fn=collate_fn)

# 4) Баланс классов (простой вариант: веса в CrossEntropy по частотам)
import numpy as np
counts = np.bincount(train_labels, minlength=2)  # counts[0], counts[1]
class_weights = torch.tensor((counts.sum() / (2.0 * np.maximum(counts, 1))), dtype=torch.float32, device=device)
criterion = nn.CrossEntropyLoss(weight=class_weights)

# 5) Оптимизируем ТОЛЬКО голову
head_params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.AdamW(head_params, lr=1e-3, weight_decay=0.01)

# 6) Тренировочный цикл (минимальный)
scaler = torch.cuda.amp.GradScaler(enabled=(device=="cuda"))  # AMP
best_val = float("inf")
for epoch in range(5):  # 3–5 эпох на линейный пробинг обычно достаточно
    model.train()
    for batch in train_loader:
        batch = {k: v.to(device) for k, v in batch.items()}
        optimizer.zero_grad(set_to_none=True)
        with torch.cuda.amp.autocast(enabled=(device=="cuda")):
            out = model(**batch)              # logits: (B, 2)
            loss = criterion(out.logits, batch["labels"])
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

    # валидация
    model.eval(); val_loss, n = 0.0, 0
    with torch.no_grad(), torch.cuda.amp.autocast(enabled=(device=="cuda")):
        for batch in val_loader:
            batch = {k: v.to(device) for k, v in batch.items()}
            out = model(**batch)
            val_loss += criterion(out.logits, batch["labels"]).item() * batch["labels"].size(0)
            n += batch["labels"].size(0)
    val_loss /= n
    print(f"epoch {epoch}: val_loss={val_loss:.4f}")
